In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import timedelta

In [2]:
#conv1 layer
fiter_size1=[5,5]
num_filters1=16
#conv2 layer
filter_size2=[5,5]
num_filters2=36
#fcl
fc_size=128
img_size=28
img_size_flat=img_size*img_size
img_shape=(img_size,img_size)
num_channels=1
num_classes=10

In [3]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.05))
def new_biases(length):
    return tf.Variable(tf.constant(0.05,shape=[length]))

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
def conv_2d(inputs,filter_size,filters,input_channels,use_pooling=True):
    weights=new_weights([filter_size,filter_size,input_channels,filters])
    bias=new_biases(filters)
    layer=tf.nn.conv2d(inputs,weights,strides=[1,1,1,1,],padding='SAME')
    layer+=bias
    if use_pooling:
        layer=tf.nn.max_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    layer=tf.nn.relu(layer)
    return layer,weights
    

In [6]:

def fcl(inputs,num_input,num_output,use_relu=True):
    weights=new_weights([num_input,num_output])
    bias=new_biases(num_output)
    layer=tf.matmul(inputs,weights)+bias
    if use_relu:
        layer=tf.nn.relu(layer)
    return layer

In [7]:
xs=tf.placeholder(tf.float32,[None,784])
ys=tf.placeholder(tf.float32,[None,10],name='y_true')
x_image=tf.reshape(xs,[-1,28,28,1])
y_true=tf.argmax(ys,axis=1)

In [8]:
conv1,weight1=conv_2d(x_image,5,16,1,use_pooling=True)

In [9]:
conv2,weight2=conv_2d(conv1,5,36,16,use_pooling=True)

In [10]:
fcl_input=tf.reshape(conv2,[-1,7*7*36])
#fcl_input
fcl_layer1=fcl(fcl_input,1764,128,use_relu=True)

In [11]:
fcl_layer2=fcl(fcl_layer1,128,10,use_relu=False)

In [12]:
fcl_layer2

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

In [13]:
y_pred=tf.nn.softmax(fcl_layer2)
y_pred_cls=tf.argmax(y_pred,axis=1)
y_pred_cls

<tf.Tensor 'ArgMax_1:0' shape=(?,) dtype=int64>

In [14]:
cross=tf.nn.softmax_cross_entropy_with_logits(logits=fcl_layer2,labels=ys)
cost=tf.reduce_mean(cross)
optimizer=tf.train.AdamOptimizer(1e-4).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [15]:
correct=tf.equal(y_pred_cls,y_true)
accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [16]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
total=0
def optimize(n):
    global total
    start=time.time()
    for i in range(total,total+n):
        x,y=mnist.train.next_batch(60)
        sess.run(optimizer,feed_dict={xs:x,ys:y})
        if i%100==0:
            acc=sess.run(accuracy,feed_dict={xs:x,ys:y})
            print("aaaaa"+str(acc))
    total+=n
    end=time.time()  
    print(" time  "+str(timedelta(seconds=int(round(end-start)))))

In [21]:
optimize(1000)

aaaaa0.6166667
aaaaa0.85
aaaaa0.93333334
aaaaa0.9
aaaaa0.9166667
aaaaa0.9166667
aaaaa0.8666667
aaaaa0.95
aaaaa0.93333334
aaaaa0.96666664
 time  0:02:20
